In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import decomposition

In [2]:
n_epochs = 30   # Number of optimization epochs
n_train = 60000    # Size of the train dataset
n_test = 10000     # Size of the test dataset
n_dim = 2         # 需要降到多少维

In [3]:
SAVE_PATH = "QNN/data/" # Data saving folder
PREPROCESS = True          # If False, skip quantum processing and load data from SAVE_PATH
PCA_DR = True               # 是否进行PCA降维处理

In [4]:
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

In [5]:
def pca_with_origin_data(ori_data, n_dim):
    #进行PCA降维
    n_sample, a, b = np.shape(ori_data)
    ori_data = ori_data.reshape(n_sample,a*b)
    pca = decomposition.PCA(n_components = n_dim)
    return pca.fit_transform(ori_data)

if PCA_DR == True: 
    test_images = pca_with_origin_data(test_images,n_dim)
    train_images = pca_with_origin_data(train_images,n_dim)
    
n_wires = int(np.ceil(np.log(n_dim) / np.log(2)))

dev = qml.device("default.qubit", wires=n_wires)

@qml.qnode(dev)

def circuit(f=None):
    qml.AmplitudeEmbedding(features=f, wires=range(n_wires), normalize=True, pad_with=0.)
    return qml.expval(qml.PauliZ(0))


def quanv(image):
    img = image.flatten()
    q_results = circuit(img)     
    return q_results


if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
      
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images_" + str(n_dim) + ".npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images_" + str(n_dim) + ".npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images_" + str(n_dim) + ".npy")
q_test_images = np.load(SAVE_PATH + "q_test_images_" + str(n_dim) + ".npy")

Quantum pre-processing of train images:
60000/60000        
Quantum pre-processing of test images:


In [6]:
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

q_model = MyModel()

#换下网络结构试试
q_history = q_model.fit(
    q_train_images,
    train_labels,
    validation_data=(q_test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

Epoch 1/30
15000/15000 - 8s - loss: 2.1462 - accuracy: 0.1964 - val_loss: 2.1253 - val_accuracy: 0.2092 - 8s/epoch - 554us/step
Epoch 2/30
15000/15000 - 8s - loss: 2.1219 - accuracy: 0.2088 - val_loss: 2.1231 - val_accuracy: 0.2119 - 8s/epoch - 524us/step
Epoch 3/30
15000/15000 - 8s - loss: 2.1209 - accuracy: 0.2056 - val_loss: 2.1222 - val_accuracy: 0.2138 - 8s/epoch - 531us/step
Epoch 4/30
15000/15000 - 8s - loss: 2.1206 - accuracy: 0.2076 - val_loss: 2.1218 - val_accuracy: 0.2178 - 8s/epoch - 529us/step
Epoch 5/30
15000/15000 - 8s - loss: 2.1206 - accuracy: 0.2097 - val_loss: 2.1223 - val_accuracy: 0.2037 - 8s/epoch - 532us/step
Epoch 6/30
15000/15000 - 8s - loss: 2.1205 - accuracy: 0.2087 - val_loss: 2.1222 - val_accuracy: 0.2157 - 8s/epoch - 530us/step
Epoch 7/30
15000/15000 - 8s - loss: 2.1206 - accuracy: 0.2070 - val_loss: 2.1216 - val_accuracy: 0.2144 - 8s/epoch - 529us/step
Epoch 8/30
15000/15000 - 8s - loss: 2.1206 - accuracy: 0.2072 - val_loss: 2.1219 - val_accuracy: 0.2203 